In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from inference_toolbox.parameter import Parameter
from inference_toolbox.model import Model
from inference_toolbox.likelihood import Likelihood
from inference_toolbox.sampler import Sampler
from inference_toolbox.visualiser import Visualiser
from inference_toolbox.domain import Domain

from data_processing.normaliser import Normaliser
from data_processing.box_gridder import BoxGridder
import os

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
log_dummy_data = pd.read_csv('data/log_dummy_data.csv')
training_data, testing_data = train_test_split(log_dummy_data, test_size=0.2)

In [3]:
excel_path = 'results/auto_generated_results/varying_Q_prior_sigma/varying_Q_prior_sigma.xlsx'


inputs = pd.read_excel(excel_path,header=[0,1,2], index_col=0)
inputs

Instance    Parameters                                              \
                   I_y                             I_z               
         Initial Value  Prior   Mu Sigma Initial Value  Prior   Mu   
1                  0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
2                  0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
3                  0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
4                  0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
5                  0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
6                  0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
7                  0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
8                  0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
9                  0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
10                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
11                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
12                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
13                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
14                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
15                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
16                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
17                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
18                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
19                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
20                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   
21                 0.1  Gamma  0.1   0.1           0.1  Gamma  0.1   

Instance                                       ...   Results            \
                                     Q         ...       I_z             
                 Sigma   Initial Value  Prior  ...      Mean     Upper   
1         1.000000e-05  30000000000000  Gamma  ...  2.867281  2.947241   
2         1.000000e-04  30000000000000  Gamma  ...  0.096990  0.131033   
3         1.000000e-03  30000000000000  Gamma  ...  0.096827  0.132188   
4         1.000000e-02  30000000000000  Gamma  ...  0.097931  0.136009   
5         1.000000e-01  30000000000000  Gamma  ...  0.098214  0.136154   
6         1.000000e+00  30000000000000  Gamma  ...  0.097878  0.137004   
7         1.000000e+01  30000000000000  Gamma  ...  0.097739  0.135618   
8         1.000000e+02  30000000000000  Gamma  ...  0.098049  0.136009   
9         1.000000e+03  30000000000000  Gamma  ...  0.098479  0.136386   
10        1.000000e+04  30000000000000  Gamma  ...  0.097875  0.136455   
11        1.000000e-06  30000000000000  Gamma  ...  0.097875  0.136455   
12        1.000000e-07  30000000000000  Gamma  ...  0.097875  0.136455   
13        1.000000e-08  30000000000000  Gamma  ...  0.097875  0.136455   
14        1.000000e-09  30000000000000  Gamma  ...  0.097875  0.136455   
15        1.000000e-10  30000000000000  Gamma  ...  0.097875  0.136455   
16        1.000000e+05  30000000000000  Gamma  ...  0.097875  0.136455   
17        1.000000e+06  30000000000000  Gamma  ...  0.097875  0.136455   
18        1.000000e+07  30000000000000  Gamma  ...  0.097875  0.136455   
19        1.000000e+08  30000000000000  Gamma  ...  0.097875  0.136455   
20        1.000000e+09  30000000000000  Gamma  ...  0.097875  0.136455   
21        1.000000e+10  30000000000000  Gamma  ...  0.097875  0.136455   

Instance                                                                      \
                                  Q                                            
          Average tau         Lower          Mean         Upper  Average tau   
1         2292.000000  2.523499e+00  2.645623e+00  2.780046e+00  3207.000000   
2            4.666667  2.177631e+13  2.867345e+13  3.614770e+13     5.000000   
3            3.000000  2.141796e+13  2.862705e+13  3.625621e+13     3.000000   
4            3.333333  2.161613e+13  2.872926e+13  3.633179e+1

In [4]:
for instance in inputs.index:

    # Initialize parameter series
    params = pd.Series({},dtype='float64')

    print('Generating instance ' +str(instance) + '...')

    # Parameter Assignment
    I_y = Parameter(name = 'I_y', init_val=np.float64(inputs[('Parameters','I_y','Initial Value')][instance]), prior_select=inputs[('Parameters','I_y','Prior')][instance].lower())
    I_y.add_prior_param("mu",np.float64(inputs[('Parameters','I_y','Mu')][instance]))
    I_y.add_prior_param("sigma",np.float64(inputs[('Parameters','I_y','Sigma')][instance]))
    params['I_y'] = I_y

    I_z = Parameter(name = 'I_z', init_val=np.float64(inputs[('Parameters','I_z','Initial Value')][instance]), prior_select=inputs[('Parameters','I_z','Prior')][instance].lower())
    I_z.add_prior_param("mu",np.float64(inputs[('Parameters','I_z','Mu')][instance]))
    I_z.add_prior_param("sigma",np.float64(inputs[('Parameters','I_z','Sigma')][instance]))
    params['I_z'] = I_z

    Q = Parameter(name = 'Q', init_val=np.float64(inputs[('Parameters','Q','Initial Value')][instance]), prior_select=inputs[('Parameters','Q','Prior')][instance].lower())
    Q.add_prior_param("mu",np.float64(inputs[('Parameters','Q','Mu')][instance]))
    Q.add_prior_param("sigma",np.float64(inputs[('Parameters','Q','Sigma')][instance]))
    params['Q'] = Q

    # Model Assignment
    model = Model(inputs[('Parameters','Model','Type')][instance].lower())
    model.add_model_param("H",np.float64(inputs[('Parameters','Model','H')][instance]))

    # Likelihood function assigmnent
    likelihood = Likelihood(inputs[('Parameters','Likelihood','Type')][instance].lower())
    likelihood.add_likelihood_param("sigma",np.float64(inputs[('Parameters','Likelihood','sigma')][instance]))
    # likelihood.add_likelihood_param("lambda_1",1)
    # likelihood.add_likelihood_param("lambda_2",0.05)

    num_samples = int(inputs[('Parameters','Sampler','N_samples')][instance])
    num_chains = int(inputs[('Parameters','Sampler','N_chains')][instance])
    thinning_rate_= int(inputs[('Parameters','Sampler','Thinning rate')][instance])


    # Initialize the sampler
    sampler = Sampler(params, model, likelihood, training_data, num_samples, show_sample_info = True, n_chains=num_chains, thinning_rate=thinning_rate_)
    hyperparams = sampler.get_hyperparams()
    params_samples, chain_samples, fields = sampler.sample_all()
    visualiser = Visualiser(testing_data, params_samples, model, sampler.hyperparams, chain_samples=chain_samples,fields = fields, previous_instance = sampler.instance, data_path = 'results/inference')
    summary = visualiser.get_summary()
    visualiser.get_traceplot()

    inputs.loc[instance,('Results','I_y', 'Lower')] = summary['overall']['I_y']['lower']
    inputs.loc[instance,('Results','I_y', 'Mean')] = summary['overall']['I_y']['mean']
    inputs.loc[instance,('Results','I_y', 'Upper')] = summary['overall']['I_y']['upper']
    inputs.loc[instance,('Results','I_y', 'Average tau')] = summary['overall']['I_y']['tau']

    inputs.loc[instance,('Results','I_z', 'Lower')] = summary['overall']['I_z']['lower']
    inputs.loc[instance,('Results','I_z', 'Mean')] = summary['overall']['I_z']['mean']
    inputs.loc[instance,('Results','I_z', 'Upper')] = summary['overall']['I_z']['upper']
    inputs.loc[instance,('Results','I_z', 'Average tau')] = summary['overall']['I_z']['tau']

    inputs.loc[instance,('Results','Q', 'Lower')] = summary['overall']['Q']['lower']
    inputs.loc[instance,('Results','Q', 'Mean')] = summary['overall']['Q']['mean']
    inputs.loc[instance,('Results','Q', 'Upper')] = summary['overall']['Q']['upper']
    inputs.loc[instance,('Results','Q', 'Average tau')] = summary['overall']['Q']['tau']

    inputs.loc[instance,('Results','Misc', 'RMSE')] = summary['RMSE']

    divergences = []
    for i in range(visualiser.num_chains):
        divergences.append(summary['chain_' + str(i+1)]['fields']['diverging'])

    inputs.loc[instance,('Results','Misc', 'Average Diverging')] = np.mean(divergences)

inputs.to_excel(excel_path)

# Sample the parameters
# params_samples, chain_samples, fields = sampler.sample_all()

Generating instance 1...
Loading Chain Samples...
Loading Samples...


/project/inference_toolbox/visualiser.py:431: RuntimeWarning: invalid value encountered in divide
  z = (xp-np.mean(xp, axis=0))/np.std(xp, axis=0)


Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 2...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 3...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 4...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 5...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 6...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 7...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance

/project/inference_toolbox/visualiser.py:431: RuntimeWarning: invalid value encountered in divide
  z = (xp-np.mean(xp, axis=0))/np.std(xp, axis=0)
/project/inference_toolbox/visualiser.py:431: RuntimeWarning: invalid value encountered in divide
  z = (xp-np.mean(xp, axis=0))/np.std(xp, axis=0)
/project/inference_toolbox/visualiser.py:431: RuntimeWarning: invalid value encountered in divide
  z = (xp-np.mean(xp, axis=0))/np.std(xp, axis=0)


Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 12...
Loading Chain Samples...
Loading Samples...


...

Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 15...
Loading Chain Samples...
Loading Samples...


/project/inference_toolbox/visualiser.py:431: RuntimeWarning: invalid value encountered in divide
  z = (xp-np.mean(xp, axis=0))/np.std(xp, axis=0)
/project/inference_toolbox/visualiser.py:431: RuntimeWarning: invalid value encountered in divide
  z = (xp-np.mean(xp, axis=0))/np.std(xp, axis=0)
/project/inference_toolbox/visualiser.py:431: RuntimeWarning: invalid value encountered in divide
  z = (xp-np.mean(xp, axis=0))/np.std(xp, axis=0)


Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 16...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 17...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 18...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 19...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 20...
Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Generating instance 21...


/project/inference_toolbox/sampler.py:61: UserWarning: There are not enough devices to run parallel chains: expected 3 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(3)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc_obj = numpyro.infer.MCMC(kernel, num_warmup=self.n_warmup, num_samples=self.n_samples, num_chains=self.n_chains, thinning=self.thinning_rate)
sample: 100%|██████████| 12500/12500 [00:04<00:00, 2692.96it/s, 3 steps of size 5.06e-01. acc. prob=0.91] 


Creating instance
